In [1]:
# Filename: run_luigi.py
import luigi, os
 

                
class PrintNumbers (luigi.Task):
    
    texte = luigi.Parameter(default = "")
 
    def requires(self):
        return []
 
    def output(self):
        self.outPut_file = self.texte + "_appel.txt"
        return luigi.LocalTarget(self.outPut_file)
 
    def run(self):
        
        with self.output().open('w') as f:
            for i in range(1, 11):
                f.write(self.texte  + "{}\n".format(i))
                

                
class SquaredNumbers(luigi.Task):
    
    texte = luigi.Parameter (default = "")
 
    def requires(self):
        return [PrintNumbers(texte = self.texte + '_toto'), PrintNumbers(texte =self.texte +'_titi')]
    
    
    def output(self):
        self.output_file = self.texte + "_resultat.txt"
        return luigi.LocalTarget(self.output_file)
    
 
    def run(self):
        
        # il faut que le nom des fichier emis par les appels changent mais on recupere le nom
        
        
        with self.input()[0].open() as fin :
            with self.output().open("w") as out :
                for line in fin:
                    print (line[:-1])
                    out.write(line)
        print ('\nSUITE\n')       
        with self.input()[1].open() as fin :
            with self.output().open("w") as out:
                
                for line in fin:
                    print (line[:-1])
                    out.write(line)
        
        print ('\nFIN\n')
        return

def detruire (name_file) :
    if os.path.exists(name_file) :
        print (name_file)
        os.remove(name_file)
    else :
        print ("######" + name_file)
    return

    
if __name__ == '__main__':
    luigi.build([SquaredNumbers(texte = "alfred"),
                 SquaredNumbers(texte = "patrick")],
                 workers=1, local_scheduler = True)
    
    print ('\nDESTRUCTION\n')
    n = "alfred_toto.txt" 
    detruire (n)
    
    n = "alfred_titi.txt"
    detruire (n)    
        
    n = "patrick_toto.txt"
    detruire (n)  
    
    n = "patrick_titi.txt"
    detruire (n)  
    
    print ("### niveau appellant")
    
    n = "alfred_resultat.txt"
    detruire (n)
    n = "patrick_resultat.txt"
    detruire (n)
    
    
    
        
    

alfred_toto1
alfred_toto2
alfred_toto3
alfred_toto4
alfred_toto5
alfred_toto6
alfred_toto7
alfred_toto8
alfred_toto9
alfred_toto10

SUITE

alfred_titi1
alfred_titi2
alfred_titi3
alfred_titi4
alfred_titi5
alfred_titi6
alfred_titi7
alfred_titi8
alfred_titi9
alfred_titi10

FIN

patrick_toto1
patrick_toto2
patrick_toto3
patrick_toto4
patrick_toto5
patrick_toto6
patrick_toto7
patrick_toto8
patrick_toto9
patrick_toto10

SUITE

patrick_titi1
patrick_titi2
patrick_titi3
patrick_titi4
patrick_titi5
patrick_titi6
patrick_titi7
patrick_titi8
patrick_titi9
patrick_titi10

FIN


DESTRUCTION

######alfred_toto.txt
######alfred_titi.txt
######patrick_toto.txt
######patrick_titi.txt
### niveau appellant
alfred_resultat.txt
patrick_resultat.txt


In [89]:
#%save Generic.py 81

In [81]:
import  os, sys, json, luigi, time, copy
from datetime import datetime

from Kernel_BE import Kernel
       
        

        
            
    
'''
###### self.get_execution() est fourni par l'utilisateur (la tache principale)
#exemple__________________def get_execution (self) :
#        
#                             self.appels = {'appel' : MyTask,
#                                           'type_repetition' : 'ID',
#                                           }
#                             return
#   permet de fournir les parametres pour les appels distribution par variables, bloc ou ID
#   si appel is None alors pas d'appel 
#
###### self.nom_entree permet à requires de fournir le nombre d appels
#                                        pour self.input [i] for i in range(nom_entree)
#                                        
# attention il est possible de choisir dans output un nom de fichier qui est le meme 
# que ceux des taches paralleles (luigi recupere les noms de fichier pour  self.input [i].open...
#______________ l'ordre semble etre respecté meme si dans cce cas cela n'a aucun interet
#
# pour les descriptions de arg_kernel et arg_travail voir les documents d' architure
# mais le format est du texte (json) compatibilité simple avec luigi sans perte de perf


le code est equivallent entre simulation et Generic

'''            


class Generic_simulation ():
    
    def __init__(self, arg_kernel = "", arg_travail = "" ) :
        
        self.arg_kernel = json.dumps (arg_kernel)
        self.arg_travail = json.dumps (arg_travail)
        
    def run (self,) :
        """
        run generique
         - pour les traces et gesion des erreurs
         - appel run_tache fournit par  la classe dependante
        """
        # on trace 
        createur = self.arg_kernel_data ['createur']
        etape = self.arg_travail_data ['etape']
        message_trace = "depart de la tache : " + self.nom_tache_luigi 
        self.kernel.log_trace (createur, etape,  message_trace)
                
        # on execute la tache
        
        try :
            self.get_kernel ()
            self.get_travail ()
            message = self.run_tache()
        except :
            createur = self.arg_kernel_data ['createur']
            etape = self.arg_travail_data ['etape']
            message = "erreur de la tache : " + self.nom_tache_luigi 
            self.kernel.log_error (createur, etape,  message)
            
        
        # on trace la fin
        createur = self.arg_kernel_data ['createur']
        etape = self.arg_travail_data ['etape']
        message_trace = "fin de la tache : " + self.nom_tache_luigi 
        self.kernel.log_trace (createur, etape,  message_trace)
        
        # on marque la fin
        # dans le cas de recuperation du travail des taches required
        # on doit jouer sur les differents self.input qui ouvre les fichiers trvud
        
        return message
    
            
    def get_kernel (self,) :
        try :
            a = self.kernel
        except :
            self.arg_kernel_data = json.loads (self.arg_kernel)
            self.kernel = Kernel (self.arg_kernel_data)
        return
    
    def get_travail (self,) :
        try :
            a = self.travail
        except :
            
            self.arg_travail_data = json.loads (self.arg_travail)
            pathDicoEvenements = self.arg_travail_data ['pathDicoEvenements']
            f = open (pathDicoEvenements, 'r')
            data = f.read()
            f.close()
            self.dicoEvenements = json.loads (data)
        
        return
    
    def run_tache (self,) :
        return
    
        
    
    def rien (self,arg_kernel = None, arg_travail = None ) : # on simule un appel nulle
        message = ""
        return message
    
    def get_execution(self,) :
        auteur = 'Generic'
        etape = "pas de garantie de fonctionnement"
        message = "la tache n'a pas de fonction get_execution  ????"
        self.kernel.log_error (auteur, etape,  message)
        raise ValueError         
        
    def requires(self):
                 
        # initialisation
        self.get_kernel()
        self.get_travail ()
        self.get_execution()
        
        
        
        appel = self.appels ['appel']
        type_repetition = self.appels ['type_repetition']
        
        if appel is None :
            self.liste_requires = []
            return []
        
        if type_repetition == 'rien' : # pour test
            self.liste_requires = [self.arg_travail_data]
            return [appel (arg_kernel = self.arg_kernel,
                          arg_travail = self.arg_travail,
                          )]
        
        if type_repetition == "variable_numerique" :
                      
            liste_variables = self.get_liste_variables_numerique ()
            
            arg_travail_local = copy.deepcopy (self.arg_travail_data)
            
            self.liste_requires = []
            resultat = []
            for nom_variable in liste_variables :
                
                arg_travail_local ['nom_variable'] = nom_variable
                arg_travail1 = json.dumps (arg_travail_local)
                travail = appel (arg_kernel = self.arg_kernel,
                               arg_travail = arg_travail1,
                              )
                resultat.append (travail)
                self.liste_requires.append (arg_travail_local)
                
                arg_travail_local = copy.deepcopy (arg_travail_local)
                continue
            
            return resultat
        
        
        
        if type_repetition == "variable" :
            
            
            liste_variables = self.get_liste_variables ()
            resultat = []
            
            arg_travail_local = copy.deepcopy (self.arg_travail_data)
            self.liste_requires = []
            for nom_variable in liste_variables :
                arg_travail_local ['nom_variable'] = nom_variable
                arg_travail1 = json.dumps (arg_travail_local)
                travail = appel (arg_kernel = self.arg_kernel,
                               arg_travail = arg_travail1,
                              )
                resultat.append (travail)
                self.liste_requires.append (arg_travail_local)
                arg_travail_local = copy.deepcopy (arg_travail_local)
                continue
            
            return resultat
        
        if type_repetition == "bloc" :
            
            liste_blocs = self.get_liste_blocs()
            resultat = []
            arg_travail_local = copy.deepcopy (self.arg_travail_data)
            self.liste_requires = []
            
            for ID_reference_min, ID_reference_max in liste_blocs :
                
                arg_travail_local ['ID_reference_min'] = ID_reference_min
                arg_travail_local ['ID_reference_max'] = ID_reference_max
                
                arg_travail1 = json.dumps (arg_travail_local)
                travail = appel (arg_kernel = self.arg_kernel,
                               arg_travail = arg_travail1,
                                )
                resultat.append (travail)
                self.liste_requires.append (arg_travail_local)
                arg_travail_local = copy.deepcopy (arg_travail_local)
                
                continue
            
            
            return resultat
                
            
        
        if type_repetition == "ID" :
                        
            liste_blocs_ID = self.get_liste_blocs_ID ()
            arg_travail_local = copy.deepcopy (self.arg_travail_data)
            resultat = []
            self.liste_requires = []
            
            for ID_reference_min, ID_reference_max in liste_blocs_ID  :
                
                arg_travail_local ['ID_reference_min'] = ID_reference_min
                arg_travail_local ['ID_reference_max'] = ID_reference_max
                
                arg_travail1 = json.dumps (arg_travail_local)
                travail = appel (arg_kernel = self.arg_kernel,
                               arg_travail = arg_travail1,
                                )
                resultat.append (travail)
                self.liste_requires.append (arg_travail_local)
                arg_travail_local = copy.deepcopy (arg_travail_local)
                continue
                
            return resultat
                  
        raise ValueError
        
        
    def get_liste_variables (self,) :
        dico = self.dicoEvenements['creation_dictionnaire']
        liste_variables = [nom_variable for nom_variable in dico.keys()]
        return liste_variables
    
    def get_liste_variables_numerique (self,) :
        dico = self.dicoEvenements['creation_dictionnaire']
        
        liste_variables = [nom_variable for nom_variable in dico.keys()]
        liste_variables_numerique = []
        for nom_variable in liste_variables :
            
            if dico [nom_variable] ['travail'] != 'quantile' :
                continue
            
            liste_variables_numerique.append(nom_variable)
            continue
        
            
        return liste_variables_numerique
    
    def get_liste_blocs_ID (self,) :
        
        path_liste_ID = self.dicoEvenements['_path_liste_ID']
        
        f = open (path_liste_ID, 'r')
        liste = f.read ()
        f.close()
        liste = json.loads(liste)
        taille_liste_ID = len(liste)
        del liste
        taille_bloc_ID = self.dicoEvenements ['_taille_bloc_ID']
        
        resultat = []
        for debut in range (0, taille_liste_ID, taille_bloc_ID ) :
            fin = debut + taille_bloc_ID
            if fin > taille_liste_ID :
                fin = taille_liste_ID
            resultat.append ([debut, fin])
        return resultat        
        
        
    def get_liste_blocs(self,) :    
        
        
        isCreationDictionnaire = self.arg_travail_data ['isCreationDictionnaire']
        if isCreationDictionnaire :
            resultat = self.dicoEvenements ['_taille_globale'] 
            return resultat
        
        dico_ajout = self.dicoEvenements ['_ajout']
        liste_ajout = [ajout.split( '_') for ajout in dico_ajout.keys()]
        numero_courant = 0
        for _, numero in liste_ajout :
            numero = int(numero)
            if numero >= numero_courant :
                numero_courant = numero
            continue
        Min, Max = dico_ajout ['_'.join(['ajout', str(numero_courant)])]
        
        taille_bloc = self.dicoEvenements ['_taille_bloc']
        
        resultat = []
        for debut in range(Min, Max, taille_bloc ) :
            fin = debut + taille_bloc
            if fin  > Max :
                fin = Max
            resultat.append ([debut, fin])
            continue
        return resultat



    
    
class Generic (luigi.Task):
        
       
    def run (self,) :
        """
        run generique
         - pour les traces et gesion des erreurs
         - appel run_tache fournit par  la classe dependante
        """
        # on trace 
        createur = self.arg_kernel_data ['createur']
        etape = self.arg_travail_data ['etape']
        message_trace = "depart de la tache : " + self.nom_tache_luigi 
        self.kernel.log_trace (createur, etape,  message_trace)
                
        # on execute la tache
        
        try :
            self.get_kernel ()
            self.get_travail ()
            message = self.run_tache()
        except :
            createur = self.arg_kernel_data ['createur']
            etape = self.arg_travail_data ['etape']
            message = "erreur de la tache : " + self.nom_tache_luigi 
            self.kernel.log_error (createur, etape,  message)
            
        
        # on trace la fin
        createur = self.arg_kernel_data ['createur']
        etape = self.arg_travail_data ['etape']
        message_trace = "fin de la tache : " + self.nom_tache_luigi 
        self.kernel.log_trace (createur, etape,  message_trace)
        
        # on marque la fin
        # dans le cas de recuperation du travail des taches required
        # on doit jouer sur les differents self.input qui ouvre les fichiers trvud
        
        return message
    
            
    def get_kernel (self,) :
        try :
            a = self.kernel
        except :
            self.arg_kernel_data = json.loads (self.arg_kernel)
            self.kernel = Kernel (self.arg_kernel_data)
        return
    
    def get_travail (self,) :
        try :
            a = self.travail
        except :
            
            self.arg_travail_data = json.loads (self.arg_travail)
            pathDicoEvenements = self.arg_travail_data ['pathDicoEvenements']
            f = open (pathDicoEvenements, 'r')
            data = f.read()
            f.close()
            self.dicoEvenements = json.loads (data)
        
        return
    
    def run_tache (self,) :
        return
    
        
    
    def rien (self,arg_kernel = None, arg_travail = None ) : # on simule un appel nulle
        message = ""
        return message
    
    def get_execution(self,) :
        auteur = 'Generic'
        etape = "pas de garantie de fonctionnement"
        message = "la tache n'a pas de fonction get_execution  ????"
        self.kernel.log_error (auteur, etape,  message)
        raise ValueError         
        
    def requires(self):
                 
        # initialisation
        self.get_kernel()
        self.get_travail ()
        self.get_execution()
        
        
        
        appel = self.appels ['appel']
        type_repetition = self.appels ['type_repetition']
        
        if appel is None :
            self.liste_requires = []
            return []
        
        if type_repetition == 'rien' : # pour test
            self.liste_requires = [self.arg_travail_data]
            return [appel (arg_kernel = self.arg_kernel,
                          arg_travail = self.arg_travail,
                          )]
        
        if type_repetition == "variable_numerique" :
                      
            liste_variables = self.get_liste_variables_numerique ()
            
            arg_travail_local = copy.deepcopy (self.arg_travail_data)
            
            self.liste_requires = []
            resultat = []
            for nom_variable in liste_variables :
                
                arg_travail_local ['nom_variable'] = nom_variable
                arg_travail1 = json.dumps (arg_travail_local)
                travail = appel (arg_kernel = self.arg_kernel,
                               arg_travail = arg_travail1,
                              )
                resultat.append (travail)
                self.liste_requires.append (arg_travail_local)
                
                arg_travail_local = copy.deepcopy (arg_travail_local)
                continue
            
            return resultat
        
        
        
        if type_repetition == "variable" :
            
            
            liste_variables = self.get_liste_variables ()
            resultat = []
            
            arg_travail_local = copy.deepcopy (self.arg_travail_data)
            self.liste_requires = []
            for nom_variable in liste_variables :
                arg_travail_local ['nom_variable'] = nom_variable
                arg_travail1 = json.dumps (arg_travail_local)
                travail = appel (arg_kernel = self.arg_kernel,
                               arg_travail = arg_travail1,
                              )
                resultat.append (travail)
                self.liste_requires.append (arg_travail_local)
                arg_travail_local = copy.deepcopy (arg_travail_local)
                continue
            
            return resultat
        
        if type_repetition == "bloc" :
            
            liste_blocs = self.get_liste_blocs()
            resultat = []
            arg_travail_local = copy.deepcopy (self.arg_travail_data)
            self.liste_requires = []
            
            for ID_reference_min, ID_reference_max in liste_blocs :
                
                arg_travail_local ['ID_reference_min'] = ID_reference_min
                arg_travail_local ['ID_reference_max'] = ID_reference_max
                
                arg_travail1 = json.dumps (arg_travail_local)
                travail = appel (arg_kernel = self.arg_kernel,
                               arg_travail = arg_travail1,
                                )
                resultat.append (travail)
                self.liste_requires.append (arg_travail_local)
                arg_travail_local = copy.deepcopy (arg_travail_local)
                
                continue
            
            
            return resultat
                
            
        
        if type_repetition == "ID" :
                        
            liste_blocs_ID = self.get_liste_blocs_ID ()
            arg_travail_local = copy.deepcopy (self.arg_travail_data)
            resultat = []
            self.liste_requires = []
            
            for ID_reference_min, ID_reference_max in liste_blocs_ID  :
                
                arg_travail_local ['ID_reference_min'] = ID_reference_min
                arg_travail_local ['ID_reference_max'] = ID_reference_max
                
                arg_travail1 = json.dumps (arg_travail_local)
                travail = appel (arg_kernel = self.arg_kernel,
                               arg_travail = arg_travail1,
                                )
                resultat.append (travail)
                self.liste_requires.append (arg_travail_local)
                arg_travail_local = copy.deepcopy (arg_travail_local)
                continue
                
            return resultat
                  
        raise ValueError
        
        
    def get_liste_variables (self,) :
        dico = self.dicoEvenements['creation_dictionnaire']
        liste_variables = [nom_variable for nom_variable in dico.keys()]
        return liste_variables
    
    def get_liste_variables_numerique (self,) :
        dico = self.dicoEvenements['creation_dictionnaire']
        
        liste_variables = [nom_variable for nom_variable in dico.keys()]
        liste_variables_numerique = []
        for nom_variable in liste_variables :
            
            if dico [nom_variable] ['travail'] != 'quantile' :
                continue
            
            liste_variables_numerique.append(nom_variable)
            continue
        
            
        return liste_variables_numerique
    
    def get_liste_blocs_ID (self,) :
        
        path_liste_ID = self.dicoEvenements['_path_liste_ID']
        
        f = open (path_liste_ID, 'r')
        liste = f.read ()
        f.close()
        liste = json.loads(liste)
        taille_liste_ID = len(liste)
        del liste
        taille_bloc_ID = self.dicoEvenements ['_taille_bloc_ID']
        
        resultat = []
        for debut in range (0, taille_liste_ID, taille_bloc_ID ) :
            fin = debut + taille_bloc_ID
            if fin > taille_liste_ID :
                fin = taille_liste_ID
            resultat.append ([debut, fin])
        return resultat        
        
        
    def get_liste_blocs(self,) :    
        
        
        isCreationDictionnaire = self.arg_travail_data ['isCreationDictionnaire']
        if isCreationDictionnaire :
            resultat = self.dicoEvenements ['_taille_globale'] 
            return resultat
        
        dico_ajout = self.dicoEvenements ['_ajout']
        liste_ajout = [ajout.split( '_') for ajout in dico_ajout.keys()]
        numero_courant = 0
        for _, numero in liste_ajout :
            numero = int(numero)
            if numero >= numero_courant :
                numero_courant = numero
            continue
        Min, Max = dico_ajout ['_'.join(['ajout', str(numero_courant)])]
        
        taille_bloc = self.dicoEvenements ['_taille_bloc']
        
        resultat = []
        for debut in range(Min, Max, taille_bloc ) :
            fin = debut + taille_bloc
            if fin  > Max :
                fin = Max
            resultat.append ([debut, fin])
            continue
        return resultat
    
   
                


In [1]:
import time, json, os
from datetime import datetime
import luigi
from Kernel_BE import Kernel

from Generic import Generic, Generic_simulation

t = time.time()


arg_kernel = {  
          "createur" : "generic", 
          "date_emetteur" : str(datetime.now()),
          "http_auth" : None,
          "timeout" : 10,
          "host" : "localhost", 
          "port" : 9200 ,
          "zipChoisi" : 'bz2',
          "index_log_error" : "trace1",
          "index_log_warning" : "trace2",
          "index_log_trace" : "trace3",
          "index_system" : "systeme",
          "isPurge_existing_index_log" : True,
          "extra_elasticsearch_args" : None,
          "trace" : False,
          "_ID_reference_base" : "000000000000000000000000000000",
    
         
                }


arg_travail = {'pathDicoEvenements' : "../data/dico_evenements.txt",
               "nom_variable" : 'test',
               'etape' : 'test_etape',
              'arg_kernel' : arg_kernel,
               'isCreationDictionnaire' : True,
               'isValeur' : True,
               'ID_reference_min' : 50,
               'ID_reference_max' : 100,
               'isTrace'  : True,
               'isID' : False,
               
              }


class MyTask (Generic_simulation) :
    
    
    #arg_kernel = luigi.Parameter(default = "")
    #arg_travail = luigi.Parameter(default = "")
    
    
    def get_execution (self) :
        self.appels = {'appel' : None,
                      'type_repetition' : 'rien',
                       }
        return
        
    def output(self):
        self.nom_tache_luigi = 'MyTask'
        return #luigi.LocalTarget("task.txt")
        
        
T = MyTask (arg_kernel = arg_kernel, arg_travail = arg_travail )

T.get_kernel()
T.get_travail()


r = T.get_liste_blocs()
voulu = [0, 192]

if r != voulu :
    print ('##### erreur get_liste_blocs avec isDictionnaire = True')
    print ('resultat =',r)
    print ('voulu =', voulu)
    print ()
    
print ('premier test en ', time.time () - t)
    
arg_travail = {'pathDicoEvenements' : "../data/dico_evenements.txt",
               "nom_variable" : 'test',
              'arg_kernel' : arg_kernel,
              'isCreationDictionnaire' : False,
              }    


T = MyTask (arg_kernel = arg_kernel, arg_travail = arg_travail )

T.get_kernel()
T.get_travail()

r = T.get_liste_blocs()

voulu = [[0, 100], [100, 192]]

if r != voulu :
    print ('##### erreur get_liste_blocs avec isDictionnaire = False')
    print ('resultat =',r)
    print ('voulu =', voulu)
    print ()
    
print ('2 ieme test ', time.time () - t)

arg_travail = {'pathDicoEvenements' : "../data/dico_evenements.txt",
               "nom_variable" : 'test',
              'arg_kernel' : arg_kernel,
              'isCreationDictionnaire' : False,
              }    


T = MyTask (arg_kernel = arg_kernel, arg_travail = arg_travail )

T.get_kernel()
T.get_travail()


r = T.get_liste_variables()
voulu = ['ID',
 'date_evenement',
 'description',
 'nomenclature_1',
 'nomenclature_2',
 'nomenclature_3',
 'nomenclature_4',
 'prix',
 'prix_panier']

if r != voulu :
    print ('##### erreur get_liste_variables')
    print ('resultat =',r)
    print ('voulu =', voulu)
    print ()
    
print ('3 ieme test en ', time.time () - t)

arg_travail = {'pathDicoEvenements' : "../data/dico_evenements.txt",
               "nom_variable" : 'test',
               'etape' : 'test_generic',
               'arg_kernel' : arg_kernel,
               'isCreationDictionnaire' : False,
              }    


T = MyTask (arg_kernel = arg_kernel, arg_travail = arg_travail )

T.get_kernel()
T.get_travail()

r = T.get_liste_blocs_ID ()
voulu = [[0, 10], [10, 20], [20, 24]]

if r != voulu :
    print ('##### erreur get_liste_blocs_ID')
    print ('resultat =',r)
    print ('voulu =', voulu)
    print ()
    
print ('4 ieme test en ', time.time () - t)

arg_travail = {'pathDicoEvenements' : "../data/dico_evenements.txt",
               "nom_variable" : 'test',
              'arg_kernel' : arg_kernel,
               'etape' : 'test etape',
              'isCreationDictionnaire' : False,
              }    


T = MyTask (arg_kernel = arg_kernel, arg_travail = arg_travail )

T.get_kernel()
T.get_travail()
r = T.output ()

r = T.requires()
voulu = []
if r != voulu :
    print ('##### erreur requires avec None')
    print ('resultat =',r)
    print ('voulu =', voulu)
    print ()
    
print ('5 ieme test en ', time.time () - t)

class MyTask1 (Generic_simulation) :
    
    
    
    
    
    def get_execution (self) :
        
        self.appels = {'appel' : MyTask,
                      'type_repetition' : 'variable',
                      }
        return
        
    def output(self):
        self.nom_tache_luigi = 'MyTask'
        return #luigi.LocalTarget("task.txt")
        
        
T = MyTask1 (arg_kernel = arg_kernel, arg_travail = arg_travail )

T.get_kernel()
T.get_travail()
r = T.output ()
r = T.requires()
if len(r) != 9 :
    print ('##### erreur require variable')
    print ('r =',r)
    
print ('6 ieme en ', time.time () - t)

class MyTask1 (Generic_simulation) :
    
    
    
    
    
    def get_execution (self) :
        
        self.appels = {'appel' : MyTask,
                      'type_repetition' : 'ID',
                       }
        return
        
    def output(self):
        #self.nom_tache_luigi = 'MyTask'
        return #luigi.LocalTarget("task.txt")
        
        
T = MyTask1 (arg_kernel = arg_kernel, arg_travail = arg_travail )

T.get_kernel()
T.get_travail()
r = T.output ()
r = T.requires()

if len(T.liste_requires) != 3  :
    print ('##### erreur require ID')
    print ('r =',T.liste_requires)

if len(r) != 3 :
    print ('##### erreur require ID')
    print ('r =',r)
print ('7 ieme test en ', time.time () - t)
    
class MyTask1 (Generic_simulation) :
    
    
    def get_execution (self) :
        
        self.appels = {'appel' : MyTask,
                      'type_repetition' : 'bloc',
                       }
        return
    def run_tache (self,) :
        print ('\n_____ OK pour run \n')
        return
        
    def output(self):
        self.nom_tache_luigi = 'MyTask'
        return #luigi.LocalTarget("task.txt")
        
        
T = MyTask1 (arg_kernel = arg_kernel, arg_travail = arg_travail )
print ('appel Mytask1  en ', time.time () - t)

T.get_kernel()
T.get_travail()
r = T.output ()
r = T.requires()
print ('init Mytask1  en ', time.time () - t)

if len(T.liste_requires) != 2  :
    print ('##### erreur require bloc')
    print ('r =',len(T.liste_requires))
    print (len(r))
    

if len(r) != 2 :
    print ('##### erreur require bloc')
    print ('r =',r)


#### reste à tester run

T.run()



class MyTask1 (Generic_simulation) :
    def get_execution (self) :
        
        self.appels = {'appel' : MyTask,
                      'type_repetition' : 'variable_numerique',
                       }
        return
    
    
    def run_tache (self,) :
        print ('\n_____ OK pour run \n')
        return
        
    def output(self):
        self.nom_tache_luigi = 'MyTask'
        return #luigi.LocalTarget("task.txt")
        
        
T = MyTask1 (arg_kernel = arg_kernel, arg_travail = arg_travail )
print ('appel Mytask1  en ', time.time () - t)

T.get_kernel()
T.get_travail()
r = T.output ()

r = T.requires ()

if len(r) != 2 :
    print ('##### erreur require variable_numerique')
    print ('r =',r)

liste = T.liste_requires
r = []
for arg_travail_data in liste :
    nom_variable = arg_travail_data ['nom_variable']
    r.append(nom_variable)
    continue
   

if r != ['prix', 'prix_panier'] :
    print ('##### erreur require variable_numerique')
    print ('r =',r)
    
    
class MyTask1 (Generic_simulation) :
    
    
    
    def run_tache (self,) :
        print ('\n_____ OK pour run \n')
        return
        
    def output(self):
        self.nom_tache_luigi = 'MyTask'
        return #luigi.LocalTarget("task.txt")
    
T = MyTask1 (arg_kernel = arg_kernel, arg_travail = arg_travail )
print ('appel Mytask1  en ', time.time () - t)


T.get_kernel()
T.get_travail()
r = T.output ()
try :
    r = T.requires()
    test = False
except:
    test = True
    
if not test :
    print ('pas de fonction execution ne provoque pas d erreur')


    
print ('end of job en ', time.time () - t)




premier test en  0.23831796646118164
2 ieme test  0.4810051918029785
3 ieme test en  0.7053341865539551
4 ieme test en  0.9574639797210693
5 ieme test en  1.1963272094726562
6 ieme en  1.4663896560668945
7 ieme test en  1.7058587074279785
appel Mytask1  en  1.7062926292419434
init Mytask1  en  1.9402198791503906

_____ OK pour run 

appel Mytask1  en  1.9806571006774902
appel Mytask1  en  2.2457659244537354
end of job en  2.5109105110168457


In [72]:
d = {'pathDicoEvenements': '../data/dico_evenements.txt', 'nom_variable': 'test', 'arg_kernel': {'createur': 'generic', 'date_emetteur': '2021-03-03 03:25:46.274306', 'http_auth': None, 'timeout': 10, 'host': 'localhost', 'port': 9200, 'zipChoisi': 'bz2', 'index_log_error': 'trace1', 'index_log_warning': 'trace2', 'index_log_trace': 'trace3', 'index_system': 'systeme', 'isPurge_existing_index_log': True, 'extra_elasticsearch_args': None, 'trace': False, '_ID_reference_base': '000000000000000000000000000000'}, 'isCreationDictionnaire': False}

In [7]:
%time (json.dumps (d))

CPU times: user 13 µs, sys: 14 µs, total: 27 µs
Wall time: 31 µs


'{"pathDicoEvenements": "../data/dico_evenements.txt", "nom_variable": "test", "arg_kernel": {"createur": "generic", "date_emetteur": "2021-03-03 03:25:46.274306", "http_auth": null, "timeout": 10, "host": "localhost", "port": 9200, "zipChoisi": "bz2", "index_log_error": "trace1", "index_log_warning": "trace2", "index_log_trace": "trace3", "index_system": "systeme", "isPurge_existing_index_log": true, "extra_elasticsearch_args": null, "trace": false, "_ID_reference_base": "000000000000000000000000000000"}, "isCreationDictionnaire": false}'

In [39]:
z['nom_variable'] = 'tets'

In [29]:
d ['nom_variable'] = t

In [ ]:
rr